# Violatio PIVOT & MERGE
* API를 통해 내려받은 차로변경 위반차량 정보를 PIVOT하여 하나로 통합하여 저장하기

In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

In [2]:
file_dir = 'data/violations_filtered'
file_list = os.listdir(file_dir)
len(file_list)

988

In [3]:
save_dir = 'data/violations_pivoted'
if not save_dir:
    os.makedirs(save_dir)

In [27]:
total_df = pd.DataFrame()

for file in tqdm(file_list):
    file_path = os.path.join(file_dir, file)
    
    date = file[0:8]
    routeNo = file[9:13]
    direction = file[14]
    
    df = pd.read_csv(file_path, encoding = 'cp949')
    
    pv = pd.pivot_table(df,
                        index = ['conzoneId', 'conzoneNm', 'laneNum', 'oTime', 'busLaneGubun'],
                        columns = ['avcCarType'],
                        values = ['totalTrafficFlow', 'violationTrafficFlow'])
    pv = pv.reset_index()
    
    pv.columns = ['conzoneId', 'conzoneNm', 'laneNum', 'oTime', 'busLaneGubun',
                  'totalTF_1', 'totalTF_2', 'totalTF_3', 'totalTF_4', 'totalTF_5', 'totalTF_6',
                  'totalTF_7', 'totalTF_8', 'totalTF_9', 'totalTF_10', 'totalTF_11', 'totalTF_12',
                  'violationTF_1', 'violationTF_2', 'violationTF_3', 'violationTF_4', 
                  'violationTF_5', 'violationTF_6', 'violationTF_7', 'violationTF_8',
                  'violationTF_9', 'violationTF_10', 'violationTF_11', 'violationTF_12']
    
    pv.insert(0, 'date', date)
    pv.insert(1, 'routeNo', routeNo)
    pv.insert(2, 'direction', direction)
    
    save_path = os.path.join(save_dir, file)
    pv.to_csv(save_path, encoding = 'cp949')
    
    total_df = pd.concat([total_df, pv])

total_df.to_csv('data/violations_merged.csv', encoding = 'cp949')

100%|████████████████████████████████████████████████████████████████████████████████| 988/988 [01:00<00:00, 16.23it/s]


In [28]:
total_df.head(3)

,date,routeNo,direction,conzoneId,conzoneNm,laneNum,oTime,busLaneGubun,totalTF_1,totalTF_2,...,violationTF_3,violationTF_4,violationTF_5,violationTF_6,violationTF_7,violationTF_8,violationTF_9,violationTF_10,violationTF_11,violationTF_12
0,20170730,0450,E,0450CZE240,여주JC-남여주IC,1,07:00,전용,15,0,...,0,0,0,0,0,0,0,0,0,0
1,20170730,0450,E,0450CZE240,여주JC-남여주IC,1,07:15,전용,7,0,...,0,0,0,0,0,0,0,0,0,0
2,20170730,0450,E,0450CZE240,여주JC-남여주IC,1,07:30,전용,9,0,...,0,0,0,0,0,0,0,0,0,0


# 12종 차종분류표
* 1종 : 16인승 미만 여객수송용 차량.
* 2종 : 16인승 이상 여객수송용 버스 형식. 2축 1단위 차량
* 3~7종 : 화물 수송용 트럭
* 8~12종 : 화물 수송용 세미 또는 풀 트레일러
---
**2~12종을 중차량(HV)으로 분류** : totalTF_HV, violationTF_HV

In [29]:
total_df['totalTF_HV'] = total_df['totalTF_2'] + total_df['totalTF_3'] + total_df['totalTF_4'] + total_df['totalTF_5'] + total_df['totalTF_6'] + total_df['totalTF_7'] + total_df['totalTF_8'] + total_df['totalTF_9'] + total_df['totalTF_10'] + total_df['totalTF_11'] + total_df['totalTF_12']

total_df['violationTF_HV'] = total_df['violationTF_2'] + total_df['violationTF_3'] + total_df['violationTF_4'] + total_df['violationTF_5'] + total_df['violationTF_6'] + total_df['violationTF_7'] + total_df['violationTF_8'] + total_df['violationTF_9'] + total_df['violationTF_10'] + total_df['violationTF_11'] + total_df['violationTF_12']

In [30]:
total_df = total_df.drop(['totalTF_2', 'totalTF_3', 'totalTF_4', 'totalTF_5', 'totalTF_6',
                         'totalTF_7', 'totalTF_8', 'totalTF_9', 'totalTF_10', 'totalTF_11', 'totalTF_12',
                         'violationTF_2', 'violationTF_3', 'violationTF_4', 'violationTF_5',
                         'violationTF_6', 'violationTF_7', 'violationTF_8', 'violationTF_9',
                         'violationTF_10', 'violationTF_11', 'violationTF_12'], 
                         axis = 'columns')

In [ ]:
total_df['totalTF'] = total_df['totalTF_1'] + total_df['totalTF_HV']
total_df['violationTF'] = total_df['violationTF_1'] + total_df['violationTF_HV']

# 위반율
total_df['total_ViolationRate'] = total_df['violationTF']/total_df['totalTF']
total_df['HV_ViolationRate'] = total_df['violationTF_HV']/total_df['totalTF_HV']

total_df = total_df.fillna(0) # NaN을 0으로 대체하여 출력

total_df.to_csv('data/violations_merged_02.csv', encoding = 'cp949')

In [32]:
total_df.head(3)

,date,routeNo,direction,conzoneId,conzoneNm,laneNum,oTime,busLaneGubun,totalTF_1,violationTF_1,totalTF_HV,violationTF_HV,totalTF,violationTF,total_ViolationRate,HV_ViolationRate
0,20170730,0450,E,0450CZE240,여주JC-남여주IC,1,07:00,전용,15,0,0,0,15,0,0.0,0.0
1,20170730,0450,E,0450CZE240,여주JC-남여주IC,1,07:15,전용,7,0,0,0,7,0,0.0,0.0
2,20170730,0450,E,0450CZE240,여주JC-남여주IC,1,07:30,전용,9,0,4,0,13,0,0.0,0.0
